In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from dotenv import load_dotenv
import pandas as pd
import os

load_dotenv("../.env")
user = os.getenv('USERR')
password = os.getenv('PASSW')

In [5]:
import json

with open('../Data2022_2/modality_data.json') as json_file:
    data = json.load(json_file)

In [38]:
op = webdriver.ChromeOptions()
op.add_argument("--headless")

driver = webdriver.Chrome('../chromedriver', options=op)
driver.get('https://www.saes.upiita.ipn.mx/')

# Finding the input tags
user_in = driver.find_element_by_css_selector('#ctl00_leftColumn_LoginUser_UserName')
pas_in = driver.find_element_by_css_selector('.passwordEntry')
captcha_in = driver.find_element_by_css_selector("#ctl00_leftColumn_LoginUser_CaptchaCodeTextBox")

# Screenshot for the captcha
driver.save_screenshot("screen.png")

# Sending the user data to the form
user_in.send_keys(user)
pas_in.send_keys(password)
captcha_in.send_keys(input("Captcha:"))
pas_in.send_keys(Keys.RETURN)

In [11]:
import unicodedata as ud

In [39]:
# Going to the ocupation page in the same session
driver.get('https://www.saes.upiita.ipn.mx/Academica/Ocupabilidad_grupos.aspx')

# Selecting actual school period
driver.find_element_by_id("ctl00_mainCopy_rblEsquema_0").click()


# Iterating over the 3 carreers
careers = {"M" : [], "B": [], "T": []}
columns_name = ["Grupo", "Semestre", "Cupo", "Inscritos", "Disponibles", "Asignatura", "id"]
for career in careers.keys():
    # Finding the carreer selector 
    career_finder = driver.find_element_by_id("ctl00_mainCopy_dpdcarrera")
    career_selector = Select(career_finder)
    # Selecting which career to find
    career_selector.select_by_value(career)
    # Getting a single string for all the web table
    ocupability_str = driver.find_element_by_id("ctl00_mainCopy_GrvOcupabilidad").text
    # Split the rows for each \n into a single string
    rows = ocupability_str.split("\n")
    # Separate in columns by the spaces between them
    ocupability_processed = [col.split(" ") for col in rows[1:]]
    # the course name gets separated, let's fix it
    for row in ocupability_processed:
        # slicing the course name
        course_name_slice = row[2:-4]
        # concatenating
        course_str = " ".join(course_name_slice)
        # deleting the separated course name string
        del row[1:-4]
        # if "PROCESO" in course_str:
        #     course_str = course_str.replace("PROCESO", "PROCESOS")
        # if "," in course_str:
        #     course_str = course_str.replace(",", "")
        # appending the concatenated string
        row.append(course_str)
        row.append(course_str+row[0])
        # row.append(ud.normalize('NFKD', course_str+row[0]).encode('ascii', errors='ignore').decode("UTF-8", "ignore"))
        # print(ud.normalize('NFKD', course_str+row[0]).encode('ascii', errors='ignore').decode("UTF-8", "ignore"))
        careers[career].append(row)
        # careers[career].append(data[row[-1]])

In [71]:
def create_id(df):
    df["id"] = df["Asignatura"] + df["Grupo"]
    return df

def concat_sort(df1, df2):
    df1 = df1.sort_values("id").reset_index(drop=True)
    df2 = df2.sort_values("id").reset_index(drop=True)
    df_complete = pd.concat([df1, df2], join="inner", axis=1)
    df_complete = df_complete.loc[:,~df_complete.columns.duplicated()]
    return df_complete

In [76]:
def to_cosmos_db(df, career_letter):
    courses_keys = df.columns.tolist()
    processed_courses = df.values.tolist()
    data = [dict(zip(courses_keys, course)) for course in processed_courses]
    api = {"courses": data, "id":career_letter}
    print(api)


In [65]:
M_ocupability = pd.DataFrame(careers["M"], columns=columns_name)
B_ocupability = pd.DataFrame(careers["B"], columns=columns_name)
T_ocupability = pd.DataFrame(careers["T"], columns=columns_name)
M_ocupability

,Grupo,Semestre,Cupo,Inscritos,Disponibles,Asignatura,id
0,1MM4,1,0,0,0,ALGEBRA LINEAL Y NUMEROS COMPLEJOS,ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MM4
1,1MM3,1,0,0,0,ALGEBRA LINEAL Y NUMEROS COMPLEJOS,ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MM3
2,1MM6,1,0,0,0,ANALISIS Y DISEÑO DE PROGRAMAS,ANALISIS Y DISEÑO DE PROGRAMAS1MM6
3,1MM7,1,0,0,0,ANALISIS Y DISEÑO DE PROGRAMAS,ANALISIS Y DISEÑO DE PROGRAMAS1MM7
4,1MV4,1,0,0,0,ANALISIS Y DISEÑO DE PROGRAMAS,ANALISIS Y DISEÑO DE PROGRAMAS1MV4
...,...,...,...,...,...,...,...
249,5MM4,5,40,0,40,TRABAJO TERMINAL I,TRABAJO TERMINAL I5MM4
250,5MM5,5,40,0,40,TRABAJO TERMINAL II,TRABAJO TERMINAL II5MM5
251,5MM6,5,40,0,40,TRABAJO TERMINAL II,TRABAJO TERMINAL II5MM6
252,5MV5,5,40,0,40,TRABAJO TERMINAL II,TRABAJO TERMINAL II5MV5


In [72]:
M_schedule = pd.read_csv("../Data2022_2/mechatronics_schedules.csv").dropna()
B_schedule = pd.read_csv("../Data2022_2/bionics_schedules.csv").dropna()
T_schedule = pd.read_csv("../Data2022_2/telematics_schedules.csv").dropna()
M_schedule

,Grupo,Asignatura,Profesor,Lun,Mar,Mie,Jue,Vie
0,1MM3,ALGEBRA LINEAL Y NUMEROS COMPLEJOS,MIRANDA SANCHEZ MICHEL GALAXIA,08:30-10:00,,08:30-10:00,,08:30-10:00
1,1MM3,MECANICA DE LA PARTICULA,PELAYO RAMOS RODRIGO,11:30-13:00,11:30-13:00,,11:30-13:00,
2,1MM4,ALGEBRA LINEAL Y NUMEROS COMPLEJOS,MIRANDA SANCHEZ MICHEL GALAXIA,,11:30-13:00,,11:30-13:00,11:30-13:00
3,1MM5,MECANICA DEL CUERPO RIGIDO,MORALES HERNANDEZ AGUSTIN,11:30-13:00,,11:30-13:00,,11:30-13:00
4,1MM5,ECUACIONES DIFERENCIALES,OJEDA MENDOZA GUALBERTO JESUS,10:00-11:30,,,08:30-10:00,10:00-11:30
...,...,...,...,...,...,...,...,...
258,5MV2,CONTROL DE MAQUINAS ELECTRICAS,HEREDIA VARGAS OMAR,,13:00-14:30,,13:00-14:30,
259,5MV3,TRABAJO TERMINAL I,HUITRON RAMÍREZ ERICK,16:00-17:30,16:00-17:30,14:30-16:00,14:30-16:00,14:30-16:00
260,5MV4,TRABAJO TERMINAL I,GUTIERREZ ESPINOZA MARIBEL,14:30-16:00,14:30-16:00,14:30-16:00,14:30-16:00,14:30-16:00
261,5MV5,TRABAJO TERMINAL II,RIVAS BONILLA ELIZABETH,13:00-14:30,13:00-14:30,13:00-14:30,13:00-14:30,13:00-14:30


In [51]:
pd.reset_option('display.max_rows', 500)

In [73]:
for career_df in [M_schedule, B_schedule, T_schedule]:
    career_df = create_id(career_df)


In [ ]:
careers = {"Mechatronics": (M_schedule, M_ocupability), "Bionics": (B_schedule, B_ocupability), "Telematics": (T_schedule, T_ocupability)}
for career_name in careers.keys():
    career_full = concat_sort(careers[career_name][0], careers[career_name][1])
    to_cosmos_db(career_full, career_name[0])

In [70]:
M_schedule = M_schedule.reset_index()
B_schedule = B_schedule.reset_index()
T_schedule = T_schedule.reset_index()
M_schedule

,index,Grupo,Asignatura,Profesor,Lun,Mar,Mie,Jue,Vie,id
0,0,1MM3,ALGEBRA LINEAL Y NUMEROS COMPLEJOS,MIRANDA SANCHEZ MICHEL GALAXIA,08:30-10:00,,08:30-10:00,,08:30-10:00,ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MM3
1,1,1MM3,MECANICA DE LA PARTICULA,PELAYO RAMOS RODRIGO,11:30-13:00,11:30-13:00,,11:30-13:00,,MECANICA DE LA PARTICULA1MM3
2,2,1MM4,ALGEBRA LINEAL Y NUMEROS COMPLEJOS,MIRANDA SANCHEZ MICHEL GALAXIA,,11:30-13:00,,11:30-13:00,11:30-13:00,ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MM4
3,3,1MM5,MECANICA DEL CUERPO RIGIDO,MORALES HERNANDEZ AGUSTIN,11:30-13:00,,11:30-13:00,,11:30-13:00,MECANICA DEL CUERPO RIGIDO1MM5
4,4,1MM5,ECUACIONES DIFERENCIALES,OJEDA MENDOZA GUALBERTO JESUS,10:00-11:30,,,08:30-10:00,10:00-11:30,ECUACIONES DIFERENCIALES1MM5
...,...,...,...,...,...,...,...,...,...,...
249,258,5MV2,CONTROL DE MAQUINAS ELECTRICAS,HEREDIA VARGAS OMAR,,13:00-14:30,,13:00-14:30,,CONTROL DE MAQUINAS ELECTRICAS5MV2
250,259,5MV3,TRABAJO TERMINAL I,HUITRON RAMÍREZ ERICK,16:00-17:30,16:00-17:30,14:30-16:00,14:30-16:00,14:30-16:00,TRABAJO TERMINAL I5MV3
251,260,5MV4,TRABAJO TERMINAL I,GUTIERREZ ESPINOZA MARIBEL,14:30-16:00,14:30-16:00,14:30-16:00,14:30-16:00,14:30-16:00,TRABAJO TERMINAL I5MV4
252,261,5MV5,TRABAJO TERMINAL II,RIVAS BONILLA ELIZABETH,13:00-14:30,13:00-14:30,13:00-14:30,13:00-14:30,13:00-14:30,TRABAJO TERMINAL II5MV5


In [64]:
M_schedule = M_schedule.dropna()
M_schedule

,Grupo,Asignatura,Profesor,Lun,Mar,Mie,Jue,Vie,id
0,1MM3,ALGEBRA LINEAL Y NUMEROS COMPLEJOS,MIRANDA SANCHEZ MICHEL GALAXIA,08:30-10:00,,08:30-10:00,,08:30-10:00,ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MM3
1,1MM3,MECANICA DE LA PARTICULA,PELAYO RAMOS RODRIGO,11:30-13:00,11:30-13:00,,11:30-13:00,,MECANICA DE LA PARTICULA1MM3
2,1MM4,ALGEBRA LINEAL Y NUMEROS COMPLEJOS,MIRANDA SANCHEZ MICHEL GALAXIA,,11:30-13:00,,11:30-13:00,11:30-13:00,ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MM4
3,1MM5,MECANICA DEL CUERPO RIGIDO,MORALES HERNANDEZ AGUSTIN,11:30-13:00,,11:30-13:00,,11:30-13:00,MECANICA DEL CUERPO RIGIDO1MM5
4,1MM5,ECUACIONES DIFERENCIALES,OJEDA MENDOZA GUALBERTO JESUS,10:00-11:30,,,08:30-10:00,10:00-11:30,ECUACIONES DIFERENCIALES1MM5
...,...,...,...,...,...,...,...,...,...
258,5MV2,CONTROL DE MAQUINAS ELECTRICAS,HEREDIA VARGAS OMAR,,13:00-14:30,,13:00-14:30,,CONTROL DE MAQUINAS ELECTRICAS5MV2
259,5MV3,TRABAJO TERMINAL I,HUITRON RAMÍREZ ERICK,16:00-17:30,16:00-17:30,14:30-16:00,14:30-16:00,14:30-16:00,TRABAJO TERMINAL I5MV3
260,5MV4,TRABAJO TERMINAL I,GUTIERREZ ESPINOZA MARIBEL,14:30-16:00,14:30-16:00,14:30-16:00,14:30-16:00,14:30-16:00,TRABAJO TERMINAL I5MV4
261,5MV5,TRABAJO TERMINAL II,RIVAS BONILLA ELIZABETH,13:00-14:30,13:00-14:30,13:00-14:30,13:00-14:30,13:00-14:30,TRABAJO TERMINAL II5MV5


In [ ]:
careers_for_json = {"Mechatronics": (M_schedule, M_ocupability), "Bionics": (B_schedule, B_ocupability), "Telematics": (T_schedule, T_ocupability)}

In [ ]:
for schedule_df in [M_schedule, B_schedule, T_schedule]:
    schedule_df = cleaning_nan(schedule_df)

for career_name in careers_for_json.keys():
    career_full = concat_sort(careers[career_name][0], careers[career_name][1])
    # to_json_api(career_full, career_name)

In [29]:
st = "PROCESO DE MA"
st = st.replace("PROCESO", "A")
st


'A DE MA'

In [8]:
data

{'ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MM1': 'EN LINEA',
 'CALCULO DIFERENCIAL E INTEGRAL1MM1': 'EN LINEA',
 'MECANICA DE LA PARTICULA1MM1': 'EN LINEA',
 'INTRODUCCION A LA MECATRONICA1MM1': 'EN LINEA',
 'HERRAMIENTAS COMPUTACIONALES1MM1': 'HIBRIDO',
 'ESTRUCTURA Y PROPIEDADES DE LOS MATERIALES1MM1': 'EN LINEA',
 'DIBUJO ASISTIDO POR COMPUTADORA1MM1': 'HIBRIDO',
 'INTRODUCCION A LA PROGRAMACION1MM1': 'EN LINEA',
 'ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MM2': 'EN LINEA',
 'MECANICA DE LA PARTICULA1MM2': 'EN LINEA',
 'INTRODUCCION A LA MECATRONICA1MM2': 'EN LINEA',
 'HERRAMIENTAS COMPUTACIONALES1MM2': 'HIBRIDO',
 'ESTRUCTURA Y PROPIEDADES DE LOS MATERIALES1MM2': 'EN LINEA',
 'DIBUJO ASISTIDO POR COMPUTADORA1MM2': 'HIBRIDO',
 'INTRODUCCION A LA PROGRAMACION1MM2': 'EN LINEA',
 'ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MM3': 'EN LINEA',
 'MECANICA DE LA PARTICULA1MM3': 'EN LINEA',
 'ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MM4': 'EN LINEA',
 'ALGEBRA LINEAL Y NUMEROS COMPLEJOS1MV1': 'EN LINEA',
 'CALCULO DIF

In [5]:
columns_name = ["Grupo", "Semestre", "Cupo", "Inscritos", "Disponibles", "Asignatura", "id"]

In [25]:
M_courses = {"courses": [], "id": "M"}
B_courses = {"courses": [], "id": "B"}
T_courses = {"courses": [], "id": "T"}
# for course in careers["M"]:
#     courses["courses"].append({k:v for k,v in zip(columns_name, course)})


In [26]:
courses_api_format = [M_courses, B_courses, T_courses]

In [19]:
courses_api_format

[{'courses': [], 'id': 'M'},
 {'courses': [], 'id': 'B'},
 {'courses': [], 'id': 'T'}]

In [ ]:
list(zip(careers.values(), courses_api_format))

In [22]:
from azure.cosmos import CosmosClient
from azure.cosmos import PartitionKey
from dotenv import load_dotenv
load_dotenv('../.env')
HOST = os.getenv('HOST')
MASTER_KEY = os.getenv('KEY')
DATABASE_ID = os.getenv('DBID')

In [24]:
client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
db = client.get_database_client(DATABASE_ID)
container_prueba = db.get_container_client("Prubea")

In [28]:
for career, for_api in zip(careers.values(), courses_api_format):
    for course in career:
        # print(course)
        for_api["courses"].append({k:v for k,v in zip(columns_name, course)})
        container_prueba.upsert_item(for_api)

In [ ]:
container_prueba.upsert_item(courses)